In [1]:
from typing import Iterable, Literal
from re import findall, match


def parser(file = 'input.txt'):
    with open(file, 'r') as file:
        broadcast = None
        lines = []
        for line in file:
            p_m, modules = line.strip('\n').split(" -> ")
            modules = modules.split(', ')
            if p_m == "broadcaster":
                broadcast = modules
                continue
            prefix, module = p_m[0], p_m[1:]
            lines.append((prefix, module, modules))
    assert broadcast is not None
    assert lines
    return broadcast, lines
        # return file.read().split('\n\n')

def solver(broadcaster: list[str], lines: list[tuple[str, str, list[str]]]):
    # lines = [*lines]
    ...
    LOW = 'low'
    HIGH = 'high'
    ON = 'on'
    OFF = 'off'
    flip_flops = {module:OFF for (prefix, module, _modules) in lines if prefix == '%'}
    conjunctions = {module:{m:LOW for (_p, m, ms) in lines if module in ms} for (prefix, module, _modules) in lines if prefix == '&'}
    a = {m:ms for (_p, m, ms) in lines}
    # print(conjunctions)
    lows = 1000
    highs = 0
    for _ in range(1000):
        print(flip_flops)
        print(conjunctions)
        queue = [(LOW, b, None) for b in broadcaster]
        for curr, module, source in queue:
            if curr == LOW:
                lows += 1
            else:
                highs += 1
            if not (module in flip_flops or module in conjunctions):
                print("SKIPPED! ", module)
                continue
            if module in flip_flops and module in conjunctions:
                raise Exception
            next_curr = None
            if module in flip_flops:
                if curr == HIGH:
                    continue 
                flip_flops[module] = ON if flip_flops[module] == OFF else OFF
                next_curr = HIGH if flip_flops[module] == ON else LOW
                for m in a[module]:
                    queue.append((next_curr, m, module))
            else:
                assert source is not None
                try:
                    conjunctions[module][source] = curr
                except Exception as e:
                    print(f"Tried changing {source=} in {module=} to {curr=}")
                    # raise e
                    continue
                next_curr = LOW if all(state == HIGH for state in conjunctions[module].values()) else HIGH
                for m in a[module]:
                    queue.append((next_curr, m, module))
        print(queue)
    print(highs, lows)
    return highs*lows






if __name__ == '__main__':
    test_p1 = solver(*parser('test_input.txt'))
    print(test_p1)
    assert test_p1 == 32000000, test_p1
    test_p1_2 = solver(*parser('test_input_2.txt'))
    print(test_p1_2)
    assert test_p1_2 == 11687500, test_p1_2
    print(solver(*parser()))

    # test_p2 = solver(parser('test_input.txt'))
    # test_p2 = solver(parser('test_input_p2.txt'))
    # print(test_p2)
    # assert test_p2 == None, test_p2
    # print(solver(*parser()))

{'a': 'off', 'b': 'off', 'c': 'off'}
{'inv': {'c': 'low'}}
[('low', 'a', None), ('low', 'b', None), ('low', 'c', None), ('high', 'b', 'a'), ('high', 'c', 'b'), ('high', 'inv', 'c'), ('low', 'a', 'inv'), ('low', 'b', 'a'), ('low', 'c', 'b'), ('low', 'inv', 'c'), ('high', 'a', 'inv')]
{'a': 'off', 'b': 'off', 'c': 'off'}
{'inv': {'c': 'low'}}
[('low', 'a', None), ('low', 'b', None), ('low', 'c', None), ('high', 'b', 'a'), ('high', 'c', 'b'), ('high', 'inv', 'c'), ('low', 'a', 'inv'), ('low', 'b', 'a'), ('low', 'c', 'b'), ('low', 'inv', 'c'), ('high', 'a', 'inv')]
{'a': 'off', 'b': 'off', 'c': 'off'}
{'inv': {'c': 'low'}}
[('low', 'a', None), ('low', 'b', None), ('low', 'c', None), ('high', 'b', 'a'), ('high', 'c', 'b'), ('high', 'inv', 'c'), ('low', 'a', 'inv'), ('low', 'b', 'a'), ('low', 'c', 'b'), ('low', 'inv', 'c'), ('high', 'a', 'inv')]
{'a': 'off', 'b': 'off', 'c': 'off'}
{'inv': {'c': 'low'}}
[('low', 'a', None), ('low', 'b', None), ('low', 'c', None), ('high', 'b', 'a'), ('high',

In [20]:
from collections import defaultdict
from math import lcm


def solver(broadcaster: list[str], lines: list[tuple[str, str, list[str]]]):
    # lines = [*lines]
    ...
    LOW = 'low'
    HIGH = 'high'
    ON = 'on'
    OFF = 'off'
    flip_flops = {module:OFF for (prefix, module, _modules) in lines if prefix == '%'}
    conjunctions = {module:{m:LOW for (_p, m, ms) in lines if module in ms} for (prefix, module, _modules) in lines if prefix == '&'}
    a = {m:ms for (_p, m, ms) in lines}
    # print(conjunctions)
    lows = 1000
    highs = 0
    counter = 0
    cycles = defaultdict(lambda: float('inf'))
    has_been_high = defaultdict(lambda: False)
    while True:
        if len(cycles) == 4:
            break
        counter += 1
        # print(flip_flops)
        # print(conjunctions)
        queue = [(LOW, b, None) for b in broadcaster]
        for curr, module, source in queue:
            if curr == LOW:
                lows += 1
            else:
                highs += 1
            if not (module in flip_flops or module in conjunctions):
                # print("SKIPPED! ", module)
                continue
            if module in flip_flops and module in conjunctions:
                raise Exception
            next_curr = None
            if module in flip_flops:
                if curr == HIGH:
                    continue 
                flip_flops[module] = ON if flip_flops[module] == OFF else OFF
                next_curr = HIGH if flip_flops[module] == ON else LOW
                for m in a[module]:
                    queue.append((next_curr, m, module))
            else:
                assert source is not None
                try:
                    conjunctions[module][source] = curr
                except Exception as e:
                    # print(f"Tried changing {source=} in {module=} to {curr=}")
                    # raise e
                    continue
                
                next_curr = LOW if all(state == HIGH for state in conjunctions[module].values()) else HIGH
                has_been_high[module] = has_been_high[module] or next_curr == HIGH
                if next_curr == LOW and module in 'nx sp cc jq' and has_been_high[module]:
                    print(next_curr, module, counter)    
                    cycles[module] = min(cycles[module], counter)
                for m in a[module]:
                    queue.append((next_curr, m, module))
        # print(queue)
    print(highs, lows)
    print(cycles)
    return lcm(*cycles.values())


solver(*parser())

low nx 3851
low nx 3851
low nx 3851
low nx 3851
low nx 3851
low nx 3851
low nx 3851
low nx 3852
low nx 3853
low nx 3853
low nx 3854
low nx 3855
low nx 3855
low nx 3856
low nx 3857
low nx 3857
low nx 3858
low nx 3859
low nx 3859
low nx 3859
low nx 3860
low nx 3861
low nx 3861
low nx 3862
low nx 3863
low nx 3863
low nx 3864
low nx 3865
low nx 3865
low nx 3866
low nx 3867
low nx 3867
low nx 3867
low nx 3868
low nx 3869
low nx 3869
low nx 3870
low nx 3871
low nx 3871
low nx 3872
low nx 3873
low nx 3873
low nx 3874
low nx 3875
low nx 3875
low nx 3875
low nx 3876
low nx 3877
low nx 3877
low nx 3878
low nx 3879
low nx 3879
low nx 3880
low nx 3881
low nx 3881
low nx 3882
low nx 3883
low nx 3883
low nx 3883
low nx 3884
low nx 3885
low nx 3885
low nx 3886
low nx 3887
low nx 3887
low nx 3888
low nx 3889
low nx 3889
low nx 3890
low nx 3891
low nx 3891
low nx 3891
low nx 3892
low nx 3893
low nx 3893
low nx 3894
low nx 3895
low nx 3895
low nx 3896
low nx 3897
low nx 3897
low nx 3898
low nx 3899
low 

241823802412393